In [2]:
import sys
import os
import time
import logging
import argparse
import numpy as np
from datetime import datetime
from joblib import Parallel, delayed

# Set the parent directory. This assumes your notebook is in the project root or a subdirectory.
# Adjust the path as necessary.
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Custom imports from your project
from estimators.statistical_descriptor import Nagler_WS
from utils.dataset_management import parse_pipeline
from utils.dataset_load import shuffle_data, DatasetLoader
from utils.fold_management import FoldManagement
from utils.label_management import LabelManagement
from utils.balance_management import BalanceManagement
from utils.figures import *
from utils.files_management import *


In [7]:
rng = np.random.RandomState(seed=442)
print(rng)

RandomState(MT19937)


In [13]:
with open('/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/parameter/config_param.yml', 'r') as file:
    tmp = yaml.safe_load(file)

tmp

{'fixed_args': {'options': {'--data_path': '/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5',
   '--fold_method': 'combinationFold',
   '--labeling_method': 'crocus',
   '--balancing_method': 'undersample',
   '--request': '(date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000))',
   '--shuffle_data': True,
   '--balance_data': False,
   '--import_list': ['from sklearn.svm import SVC',
    'from sklearn.neighbors import KNeighborsClassifier',
    'from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier',
    'from sklearn.linear_model import LogisticRegression',
    'from sklearn.neural_network import MLPClassifier',
    'from estimators.statistical_descriptor import *',
    'from estimators.band_transform import *'],
   '--pipeline': [[['KNN_direct'],
     ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}],
     ['BandTransformer', {'bands':

In [37]:
import_list = tmp["fixed_args"]["options"]["--import_list"]
pipeline = tmp["fixed_args"]["options"]["--pipeline"]
pipeline

[[['KNN_direct'],
  ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}],
  ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}],
  ['Hist_SAR'],
  ['KNeighborsClassifier', {'n_neighbors': 50}]],
 [['RandomForest_direct'],
  ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}],
  ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}],
  ['Hist_SAR'],
  ['RandomForestClassifier', {'n_estimators': 200}, {'criterion': 'entropy'}]],
 [['MLP_direct'],
  ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}],
  ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}],
  ['Hist_SAR'],
  ['MLPClassifier', {'alpha': 0.01}]],
 [['SVMrbf_direct'],
  ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}],
  ['BandTransformer', {'bands': [0, 1, 2, 3]}, {'transformations': []}],
  ['Hist_SAR'],
  ['SVC', {'kernel': 'rbf'}, {'probability': True}]],
 [['LogisticR_direct'],
  ['BandSelector', {'bands': [0, 1, 2, 3, 4, 5, 6, 7, 8]}],
  ['BandTrans

In [9]:
load_h5("/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_3/group_1/models/AdaBoost_direct/AdaBoost_direct_fold1.h5")

Pipeline(memory='.cache',
         steps=[('BandSelector',
                 BandSelector(bands=[0, 1, 2, 3, 4, 5, 6, 7, 8])),
                ('BandTransformer',
                 BandTransformer(bands=[0, 1, 2, 3], transformations=[])),
                ('Hist_SAR', Hist_SAR()),
                ('AdaBoostClassifier',
                 AdaBoostClassifier(n_estimators=200,
                                    random_state=RandomState(MT19937) at 0x73949AC21F40))],
         verbose=True)

In [10]:
load_h5("/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_3/group_1/models/KNN_direct/KNN_direct_fold0.h5")

Pipeline(memory='.cache',
         steps=[('BandSelector',
                 BandSelector(bands=[0, 1, 2, 3, 4, 5, 6, 7, 8])),
                ('BandTransformer',
                 BandTransformer(bands=[0, 1, 2, 3], transformations=[])),
                ('Hist_SAR', Hist_SAR()),
                ('KNeighborsClassifier', KNeighborsClassifier(n_neighbors=50))],
         verbose=True)

In [11]:
load_h5("/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_3/group_1/models/LogisticR_direct/LogisticR_direct_fold0.h5")

Pipeline(memory='.cache',
         steps=[('BandSelector',
                 BandSelector(bands=[0, 1, 2, 3, 4, 5, 6, 7, 8])),
                ('BandTransformer',
                 BandTransformer(bands=[0, 1, 2, 3], transformations=[])),
                ('Hist_SAR', Hist_SAR()),
                ('LogisticRegression',
                 LogisticRegression(random_state=RandomState(MT19937) at 0x73949AC22340))],
         verbose=True)

In [12]:
load_h5("/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_3/group_1/models/MLP_direct/MLP_direct_fold0.h5")

Pipeline(memory='.cache',
         steps=[('BandSelector',
                 BandSelector(bands=[0, 1, 2, 3, 4, 5, 6, 7, 8])),
                ('BandTransformer',
                 BandTransformer(bands=[0, 1, 2, 3], transformations=[])),
                ('Hist_SAR', Hist_SAR()),
                ('MLPClassifier',
                 MLPClassifier(alpha=0.01,
                               random_state=RandomState(MT19937) at 0x73949AC22440))],
         verbose=True)

In [13]:
load_h5("/home/listic/Bureau/cortes_stage/ML-WetSnowSAR_pipeline_stage/pipeline/results/pipeline/run_3/group_1/models/RandomForest_direct/RandomForest_direct_fold0.h5")

Pipeline(memory='.cache',
         steps=[('BandSelector',
                 BandSelector(bands=[0, 1, 2, 3, 4, 5, 6, 7, 8])),
                ('BandTransformer',
                 BandTransformer(bands=[0, 1, 2, 3], transformations=[])),
                ('Hist_SAR', Hist_SAR()),
                ('RandomForestClassifier',
                 RandomForestClassifier(n_estimators=200,
                                        random_state=RandomState(MT19937) at 0x73949AC22C40))],
         verbose=True)